<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/compare_extractive_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 
!pip install rouge-score
!pip install import-ipynb

In [2]:
import pickle
from rouge_score import rouge_scorer
import import_ipynb
from nltk.corpus import stopwords  
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# load in functions from extract_summarization notebook
%cd "drive/MyDrive/Colab Notebooks"
import extractive_summarization
%cd ..

/content/drive/MyDrive/Colab Notebooks
importing Jupyter notebook from extractive_summarization.ipynb
/content/drive/MyDrive


## Calculate P-Value with Paired Bootstrap Test

For best configuration for each evaluation metric, compare the 3 model types: which model is the best and what is the p-value?
1. Calculate difference in stat performance (recall etc.)
2. Generate N bootstrapped samples of data 
3. Train on bootstrapped data
4. Calculate difference in performance on bootstraped data
5. Count percent of replicate diffs that are >= 2 * original diff = p-value  
  - Null hypothesis is that there is no difference and the true effect size is original diff (data happens to be biased towards one model)
  - If see a lot of replicated diffs >= 2 * original diff, then null is true and there is no difference between the models

In [5]:
def paired_bootstrap(evals, models, configs, model1, model2):
  embeddings = extractive_summarization.load_embeddings()

  pvalue_dict = {}
  for metric in [('fmeasure', False), ('fmeasure', True), ('precision', False), ('precision', True), ('recall', False), ('recall', True)]:

    # which model is better and by how much
    if evals[model1][metric]['mean'] > evals[model2][metric]['mean']:
      better_model = model1
      other_model = model2
    else:
      better_model = model2
      other_model = model1
    diff =  evals[better_model][metric]['mean'] - evals[other_model][metric]['mean']
    print(better_model, diff)

    #bs_results_sd = []
    gt_diff = 0
    lt_diff = 0
    for i in range(2): # TODO need more samples...once have more full models, see how big standard deviation of means are with BS = 10, 20 etc. Need to go bigger? 
                        # definitely more than 10 so can get fractions beyond 0 and 0.1
      print('BS', i)
      # generate bootstrap samples 
      bs_sample = {}
      bs_sample[model1] = models[model1][metric].sample(n = len(models[model1][metric]), replace = True)
      bs_sample[model2] = models[model2][metric].sample(n = len(models[model2][metric]), replace = True)

      # retrain both models on bootstrap samples with the current config
      bs_results = {}
      for m in [model1, model2]:
        config = tuple(config_dict[m][metric].strip('(').strip(')').replace("'", "").split(', '))
        if 'baseline' in config:
          tfidf, feature_array = extractive_summarization.corpus_tfidf(bs_sample[m])
        else:
          tfidf = ''
          feature_array = ''
        eval_results, _ = extractive_summarization.train_config_loop(bs_sample[m], tfidf, feature_array, embeddings, stop_words, 
                                                                     [config], eval_only = True)
        bs_results[m] = eval_results[str(config)][metric]['mean']
    
      #bs_results_sd.append(eval_results[config][metric]['mean'])
    #return bs_results_sd
      # find difference in relevant stat
      diff_bs = bs_results[better_model] - bs_results[other_model]
      print(diff_bs)
      if diff_bs >= 2*diff:
        gt_diff += 1
      else:
        lt_diff += 1
      
    # calculate p value
    pvalue = gt_diff / (gt_diff + lt_diff)
    pvalue_dict[metric] = (better_model, pvalue)

  return pvalue_dict

In [6]:
model_dict = {}
eval_dict = {}
config_dict = {}
for model in ['lsa']: # 'baseline']: #'textrank']:
  with open('/content/drive/MyDrive/data/trained_model_' + model + '.pkl', 'rb') as f:
    load = pickle.load(f)
    eval_dict[model] = load[1]
    model_dict[model] = load[2]
    config_dict[model] = load[3]

In [7]:
pvalue_dict = paired_bootstrap(eval_dict, model_dict, config_dict, 'lsa', 'lsa')

lsa 0.0
BS 0
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
0.0
BS 1
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
0.0
lsa 0.0
BS 0
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
0.0
BS 1
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
('lsa', 'nostop', 'no_stemlemma', 'bow', 'binary', 'tf', 'trigram', 'num_words_gt')
0.0
lsa 0.0
BS 0
('lsa', 'stopwords', 'stem', 'bow', 'binary', 'tfidf', 'unigram', 'num_sentences')
('lsa', 'stopwords', 'stem', 'bow', 'binary', 'tfidf', 'unigram', 'num_sentences')
0.0
BS 1
('lsa', 'stopwords', 'stem', 'bow', 'binary', 'tfidf', 'unigram', 'num_sentences')
('lsa

In [8]:
# TODO stats on types of summaries generated by each model and for best of each metric. Ex: recall best summaries longer than precision summaries? etc.
# problems with models include that sentences can appear in odd orders. BUT our human content creator can rearrange - care more about getting the right info in front of them.
  # thus prefer longer summaries? care more about recall than precision?